In [1]:
from DataLoader import DataLoader
from Anonymisation import Anonymisation
from Consistenter import Consistenter
from GUM import GraduallyUpdateMethod
from PostProcessor import RecordPostprocessor

In [2]:
dl = DataLoader('AccidentalDrugDeaths/accidental_drug_deaths.csv', 'AccidentalDrugDeaths/data.yaml', 'AccidentalDrugDeaths/column_info.json', 'AccidentalDrugDeaths/loading_data.json')
dl.data_loader()
dl.all_indifs(dl.private_data)
pass

,Date,Age,Sex,Race,Residence State,Death County,Location,Location if Other,Injury County,Injury State,...,Tramad,Morphine (Not Heroin),Hydromorphone,Xylazine,Opiate NOS,Any Opioid,CardioCondition,RespiratoryCondition,ObesityCondition,DiabetesCondition
0,0,2,0,6,0,1,5,6,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,4,1,6,0,4,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,1,6,0,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,4,1,6,0,3,6,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,4,1,6,0,6,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
anon = Anonymisation(10,3e-11)
anon.anonymiser(dl)
pass

epsilon: 10, delta: 3e-11, rho: 0.8616407135665874
one-way sigma: 13.412284072685887
indif sigma: 1.395e-08
multi-way sigma: 8.643617003684135


In [4]:
c = Consistenter(anon, dl.all_attrs)
c.make_consistent(iterations = 5)
pass

/opt/anaconda3/envs/privsyntest/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in divide
  result = getattr(ufunc, method)(*inputs, **kwargs)
/opt/anaconda3/envs/privsyntest/lib/python3.9/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in multiply
  result = getattr(ufunc, method)(*inputs, **kwargs)


Iteration 1 of 5 to make the marginals consistent and normalise them
Iteration 2 of 5 to make the marginals consistent and normalise them
Iteration 3 of 5 to make the marginals consistent and normalise them
Iteration 4 of 5 to make the marginals consistent and normalise them
Iteration 5 of 5 to make the marginals consistent and normalise them


In [5]:
gum = GraduallyUpdateMethod(dl, c)
gum.initialiser(view_iterations = 100)
syn_data = gum.synthesize(iterations = 100, num_records = int(c.num_synthesize_records))
pass

Iteration 1 of 100 completed to consist the marginal views
Iteration 2 of 100 completed to consist the marginal views
Iteration 3 of 100 completed to consist the marginal views
Iteration 4 of 100 completed to consist the marginal views
Iteration 5 of 100 completed to consist the marginal views
Iteration 6 of 100 completed to consist the marginal views
Iteration 7 of 100 completed to consist the marginal views
Iteration 8 of 100 completed to consist the marginal views
Iteration 9 of 100 completed to consist the marginal views
Iteration 10 of 100 completed to consist the marginal views
Iteration 11 of 100 completed to consist the marginal views
Iteration 12 of 100 completed to consist the marginal views
Iteration 13 of 100 completed to consist the marginal views
Iteration 14 of 100 completed to consist the marginal views
Iteration 15 of 100 completed to consist the marginal views
Iteration 16 of 100 completed to consist the marginal views
Iteration 17 of 100 completed to consist the marg

/Users/finalsukjin/Desktop/Final PrivSyn/GUM.py:45: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  self.synthesize_records(attrs, domains, clusters, num_records)


Iteration 53 of 100 completed to generate the dataset using the noisy marginals
Iteration 54 of 100 completed to generate the dataset using the noisy marginals
Iteration 55 of 100 completed to generate the dataset using the noisy marginals
Iteration 56 of 100 completed to generate the dataset using the noisy marginals
Iteration 57 of 100 completed to generate the dataset using the noisy marginals
Iteration 58 of 100 completed to generate the dataset using the noisy marginals
Iteration 59 of 100 completed to generate the dataset using the noisy marginals
Iteration 60 of 100 completed to generate the dataset using the noisy marginals
Iteration 61 of 100 completed to generate the dataset using the noisy marginals
Iteration 62 of 100 completed to generate the dataset using the noisy marginals
Iteration 63 of 100 completed to generate the dataset using the noisy marginals
Iteration 64 of 100 completed to generate the dataset using the noisy marginals
Iteration 65 of 100 completed to generat

In [6]:
processor_priv = RecordPostprocessor(dl.private_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
processor_pub = RecordPostprocessor(syn_data, dl.configpath, dl.datainfopath, dl.decode_mapping)
priv_data = processor_priv.post_process()
pub_data = processor_pub.post_process()

## Marginal Difference Score

In [7]:
t = 0
for i in range(len(dl.all_attrs)):
    for j in range(i+1, len(dl.all_attrs)):
        attr1 = dl.all_attrs[i]
        attr2 = dl.all_attrs[j]
        priv_marg = dl.two_way_marginal(attr1, attr2, dl.private_data)/len(dl.private_data)
        pub_marg = dl.two_way_marginal(attr1, attr2, syn_data)/len(syn_data)
        t += (priv_marg - pub_marg).abs().sum().sum()
t/(len(dl.all_attrs)*(len(dl.all_attrs)-1))

0.12778130327845041